# ACTモデルの訓練

## 概要

作成したデータセットを使用し、[ACTモデル][1]を訓練する。

[ACTモデルの解説][2]

[1]: https://huggingface.co/docs/lerobot/en/act
[2]: https://github.com/hayatoshibahara/act/blob/main/main.ipynb

## 環境構築

In [ ]:
# 環境変数の取得

try: 
    # Google Colabの場合
    from google.colab import userdata
    IS_COLAB = True
    HF_USER = userdata.get("HF_USER")
    HF_TOKEN = userdata.get("HF_TOKEN")
    WANDB_API_KEY = userdata.get("WANDB_API_KEY")
    import os
    os.environ["HF_USER"] = HF_USER
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
except:
    # ローカル環境の場合
    from dotenv import load_dotenv
    IS_COLAB = False
    import os
    load_dotenv()
    HF_USER = os.getenv("HF_USER")
    HF_TOKEN = os.getenv("HF_TOKEN")
    WANDB_API_KEY = os.getenv("WANDB_API_KEY")

assert HF_USER
assert HF_TOKEN
assert WANDB_API_KEY

In [ ]:
# Conda環境のセットアップ

try:
    import condacolab
except:
    %pip install -q condacolab
    import condacolab
    condacolab.install()

# 初回はここで再起動

In [ ]:
# Wandbのログイン

import wandb
wandb.login(key=WANDB_API_KEY)

In [ ]:
## LeRobotのインストール

if IS_COLAB:
    import os
    if not os.path.exists("/content/lerobot"):
        !git clone -b v0.4.2 --depth 1 https://github.com/huggingface/lerobot.git
    %cd /content/lerobot
    %pip install -e .
else:
    import os
    if not os.path.exists("/workspace/lerobot"):
        !git clone -b v0.4.2 --depth 1 https://github.com/huggingface/lerobot.git
    %cd /workspace/lerobot
    %pip install -e .

import lerobot
%pip show lerobot

# 初回はここで強制再起動

## 訓練

In [ ]:
from datetime import datetime
now = datetime.now()
DATETIME = now.strftime("%Y-%m-%d-%H-%M-%S")
os.environ["DATETIME"] = DATETIME

# 使用するデータセット名（注意: 要変更）
DATASET_REPO_NAME = "your-dataset-repo-name"
os.environ["DATASET_REPO_NAME"] = DATASET_REPO_NAME

OUTPUT_DIR = f"{DATASET_REPO_NAME}_act_{DATETIME}"
JOB_NAME = f"{DATASET_REPO_NAME}_act_{DATETIME}"
POLICY_REPO_NAME = f"{DATASET_REPO_NAME}_act_{DATETIME}"

# バッチサイズ（注意: 要調整）
# メモリ不足が発生する場合は下げ、逆に余裕がある場合は上げる
# 1, 2, 4, 8, 16, 32, 64...など2の累乗が一般的
# 例: A100の場合はバッチサイズ8で5/40GBで余裕あり、バッチサイズ32でも動作しそう
# 例: H100の場合はバッチサイズ64で30.2GB/80GBで余裕があり、バッチサイズ128でも動作しそう
BATCH_SIZE = 64

COMMAND = f"lerobot-train \
    --batch_size={BATCH_SIZE} \
    --job_name={JOB_NAME} \
    --log_freq=100 \
    --num_workers=0 \
    --resume=false \
    --save_freq=1000 \
    --steps=200_000 \
    --output_dir={OUTPUT_DIR} \
    --dataset.repo_id={HF_USER}/{DATASET_REPO_NAME} \
    --policy.device=cuda \
    --policy.repo_id={HF_USER}/{POLICY_REPO_NAME} \
    --policy.type=act \
    --wandb.enable=true"

print(COMMAND)

# コマンドを実行
# バッチサイズを上げると1ステップあたりの処理時間が長くなるが、学習の進みは早くなる
# 学習の進捗はWandbのダッシュボードで確認し、train/lossが収束したら学習を停止する
!$COMMAND

# より汎化性能を高めたい場合はdataset.image_transformsオプションを使用して画像変換を追加する
# https://huggingface.co/docs/lerobot/en/lerobot-dataset-v3#image-transforms

コマンドライン引数の説明は、`src/lerobot/configs/train.py`にある。

一般的な設定

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `dataset` | `DatasetConfig` | (必須) | データセット設定。 |
| `env` | `envs.EnvConfig \| None` | `None` | 環境設定。 |
| `policy` | `PreTrainedConfig \| None` | `None` | ポリシー設定。 |
| `output_dir` | `Path \| None` | `None` | 実行時の全出力を保存するディレクトリ。<br>`resume`を`True`に設定しない限り、同じディレクトリで別のトレーニングセッションを実行すると、内容は上書きされます。 |
| `job_name` | `str \| None` | `None` | ジョブ名。 |
| `resume` | `bool` | `False` | 以前の実行を再開するかどうか。<br>再開するには、`output_dir`が少なくとも1つのチェックポイントを含む既存の実行ディレクトリである必要があります。<br>再開時、コマンドライン引数に関わらず、チェックポイント内の設定がデフォルトで使用されることに注意してください。 |
| `seed` | `int \| None` | `1000` | トレーニング（モデル初期化、データセットのシャッフルなど）および評価環境で使用されるシード値。 |
| `num_workers` | `int` | `4` | データローダーで使用するワーカー数。 |
| `batch_size` | `int` | `8` | バッチサイズ。 |
| `steps` | `int` | `100_000` | トレーニングを行う総ステップ数。 |
| `eval_freq` | `int` | `20_000` | 評価を実行する頻度（ステップ数）。 |
| `log_freq` | `int` | `200` | ログを記録する頻度（ステップ数）。 |
| `save_checkpoint` | `bool` | `True` | チェックポイントを保存するかどうか。 |
| `save_freq` | `int` | `20_000` | チェックポイントを保存する頻度（ステップ数）。最後のトレーニングステップ後にも保存されます。 |
| `use_policy_training_preset` | `bool` | `True` | ポリシー固有のトレーニングプリセット（推奨されるオプティマイザ設定など）を使用するかどうか。 |
| `optimizer` | `OptimizerConfig \| None` | `None` | オプティマイザ設定。 |
| `scheduler` | `LRSchedulerConfig \| None` | `None` | 学習率スケジューラ設定。 |
| `eval` | `EvalConfig` | `EvalConfig()` | 評価設定。 |
| `wandb` | `WandBConfig` | `WandBConfig()` | Weights & Biasesの設定。 |
| `checkpoint_path` | `Path \| None` | `None` | （内部使用）再開時に使用されるチェックポイントのパス。初期化時は `None`。 |
| `rename_map` | `dict[str, str]` | `{}` | 観測データ（画像や状態のキー）を上書きするためのリネームマップ。 |

データセットの設定（DatasetConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `repo_id` | `str` | (必須) | データセットのリポジトリID。データセットのリストを指定することもでき、その場合 `train.py` がそれらを連結します。<br>注意: データセット間で共通するデータキーのみが保持されます。各データセットには、返されるアイテムに "dataset_index" を挿入する追加の変換処理が適用され、インデックスの割り当てはデータセットの指定順に基づいて行われます。 |
| `root` | `str \| None` | `None` | データセットが保存されるルートディレクトリ（例: `dataset/path`）。 |
| `episodes` | `list[int] \| None` | `None` | 使用するエピソードのリスト。 |
| `image_transforms` | `ImageTransformsConfig` | `ImageTransformsConfig()` | 画像変換設定。 |
| `revision` | `str \| None` | `None` | データセットのリビジョン（バージョン）。 |
| `use_imagenet_stats` | `bool` | `True` | ImageNetの統計情報（平均・標準偏差）を使用するかどうか。 |
| `video_backend` | `str` | *システム依存* | 動画の読み込みに使用するバックエンド（例: `pyav`, `opencv`）。 |
| `streaming` | `bool` | `False` | ストリーミングモード（データを全てダウンロードせずに逐次読み込む）を使用するかどうか。 |

環境設定（EnvConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `task` | `str \| None` | `None` | タスクの名称。 |
| `fps` | `int` | `30` | 1秒あたりのフレーム数（FPS）。 |
| `features` | `dict[str, PolicyFeature]` | `{}` | 環境が提供する特徴量（観測やアクションなど）の定義。 |
| `features_map` | `dict[str, str]` | `{}` | 特徴量の名前マッピング。 |
| `max_parallel_tasks` | `int` | `1` | 並行して実行するタスクの最大数。 |
| `disable_env_checker` | `bool` | `True` | Gymの環境チェッカーを無効にするかどうか。 |

事前学習済みモデルの設定（PreTrainedConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `n_obs_steps` | `int` | `1` | ポリシーに渡す観測データの環境ステップ数（現在のステップと、過去に遡る追加ステップを含みます）。 |
| `input_features` | `dict[str, PolicyFeature]` | `{}` | ポリシーへの入力データの定義。 |
| `output_features` | `dict[str, PolicyFeature]` | `{}` | ポリシーからの出力データの定義。 |
| `device` | `str \| None` | `None` | 使用するデバイス（例: "cuda", "cuda:0", "cpu", "mps"）。 |
| `use_amp` | `bool` | `False` | トレーニングおよび評価でAutomatic Mixed Precision (AMP)を使用するかどうかを決定します。AMPを使用すると、自動勾配スケーリングが有効になります。 |
| `push_to_hub` | `bool` | `True` | モデルをHugging Face Hubにアップロードするかどうか。 |
| `repo_id` | `str \| None` | `None` | Hugging Face HubのリポジトリID。 |
| `private` | `bool \| None` | `None` | Hugging Face Hubのプライベートリポジトリにアップロードするかどうか。 |
| `tags` | `list[str] \| None` | `None` | Hub上のポリシーに追加するタグのリスト。 |
| license | `str \| None` | `None` | ライセンス情報。 |
| `pretrained_path` | `Path \| None` | `None` | HubでホストされているモデルのリポジトリID、または `Policy.save_pretrained` を使用して保存された重みを含むディレクトリへのパス。<br>指定されない場合、ポリシーはスクラッチから初期化されます。 |

オプティマイザの設定（OptimizerConfig）

| フィールド名 | 型 | 説明 |
| :--- | :--- | :--- |
| `lr` | `float` | 学習率（Learning Rate）。 |
| `weight_decay` | `float` | 重み減衰（Weight Decay）。過学習を防ぐための正則化項です。 |
| `grad_clip_norm` | `float` | 勾配クリッピングのノルム値。勾配爆発を防ぐために勾配のノルムをこの値に制限します。 |

学習スケージューラの設定（LRSchedulerConfig）

| フィールド名 | 型 | 説明 |
| :--- | :--- | :--- |
| `num_warmup_steps` | `int` | ウォームアップステップ数。学習率を0から初期学習率まで徐々に上げていく期間のステップ数です。 |

Weight and Biases の設定（WandbConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `enable` | `bool` | `False` | Weights & Biases (WandB) のログ記録を有効にするかどうか。 |
| `disable_artifact` | `bool` | `False` | `training.save_checkpoint=True` であっても、アーティファクトの保存を無効にする場合は `True` に設定します。 |
| `project` | `str` | `"lerobot"` | WandBのプロジェクト名。 |
| `entity` | `str \| None` | `None` | WandBのエンティティ（ユーザー名またはチーム名）。 |
| `notes` | `str \| None` | `None` | 実行に関するメモ。 |
| `run_id` | `str \| None` | `None` | 実行ID。以前の実行を特定して再開する場合などに使用します。 |
| `mode` | `str \| None` | `None` | 実行モード（許可される値: `'online'`, `'offline'`, `'disabled'`）。デフォルトは `'online'` です。 |

## トラッキング

Wandbで訓練損失をトラッキングできる。

損失の変動が落ち着くまで訓練を続ける（以下の場合は20kで早期終了する）。

バッチサイズ8、50エピソードのデータセットで、A100を使用して訓練すると1時間20分程度で完了する。

![](asset/trained.png)

## 非同期アップロード

Google Colabでターミナルを開き、訓練中のモデルをアップロードできる。

```sh
# トークンを設定
HF_USER="..."
HF_TOKEN="..."

# 第2引数はアップロード先のリポジトリIDを選択する
# 第3引数はアップロードしたいモデルのパスを選択する
hf upload \
    --token $HF_TOKEN \
    $HF_USER/grab-a-cube_act_2026-01-17-06-56-00_001000 \
    lerobot/your_model_name/checkpoints/$NUM_STEPS/pretrained_model
```

## 事前学習済みモデルのファインチューニング

```sh
!lerobot-train \
    --dataset.repo_id=EngineerCafeJP/record-test-2025-08-26-21-21-00 \
    --policy.path=EngineerCafeJP/act-so101-test \
    --policy.push_to_hub=true \
    --policy.repo_id=EngineerCafeJP/act-so101-test-2025-08-26-21-21-00 \
    --output_dir=act_output \
    --job_name=act_so101_test_2025-08-26-21-21-00 \
    --resume=false \
    --num_workers=0 \
    --batch_size=8 \
    --steps=20000 \
    --save_freq=2000 \
    --wandb.enable=true
```